In [ ]:
# =======================
# IMPORT LIBRARIES
# =======================
from flask import Flask, render_template, request
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os

# =======================
# FLASK APP
# =======================
app = Flask(__name__)

# =======================
# CNN MODEL CREATION
# =======================
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# =======================
# DATA PREPROCESSING
# =======================
datagen = ImageDataGenerator(rescale=1./255)

# (Assume dataset exists — exam logic is enough)
train_data = datagen.flow_from_directory(
    'dataset/train',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

# =======================
# TRAIN CNN MODEL
# =======================
model.fit(train_data, epochs=5)

# =======================
# FLASK ROUTES
# =======================
@app.route('/')
def home():
    return '''
    <h2>CNN Image Classification</h2>
    <form action="/predict" method="post" enctype="multipart/form-data">
        <input type="file" name="image">
        <input type="submit" value="Predict">
    </form>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    img = request.files['image']
    img_path = "uploaded_image.jpg"
    img.save(img_path)

    img = load_img(img_path, target_size=(64,64))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0

    prediction = model.predict(img)

    if prediction[0][0] > 0.5:
        result = "Class 1"
    else:
        result = "Class 0"

    return f"<h2>Prediction Result: {result}</h2>"

# =======================
# RUN FLASK APP
# =======================
if __name__ == '__main__':
    app.run(debug=True)
